In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1528227,2021-08-04,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1528228,2021-08-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1528229,2021-08-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1528230,2021-08-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
40163,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
40165,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
40167,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
40169,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
40171,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
934081,2021-08-04,Arkansas,Arkansas,5001,2623,43.00,5000,Arkansas,AR,Arkansas,State,3017804
934083,2021-08-05,Arkansas,Arkansas,5001,2644,43.00,5000,Arkansas,AR,Arkansas,State,3017804
934085,2021-08-06,Arkansas,Arkansas,5001,2663,43.00,5000,Arkansas,AR,Arkansas,State,3017804
934087,2021-08-07,Arkansas,Arkansas,5001,2695,44.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1528227,2021-08-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1528228,2021-08-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1528229,2021-08-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1528230,2021-08-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-08-08') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
565,2021-08-08,Snohomish,Washington,44286,625.00,53061,WA,County,822083,5387.05,76.03
1128,2021-08-08,Cook,Illinois,568822,11147.00,17031,IL,County,5150233,11044.59,216.44
1690,2021-08-08,Orange,California,287661,5153.00,6059,CA,County,3175692,9058.21,162.26
2251,2021-08-08,Maricopa,Arizona,595609,10546.00,4013,AZ,County,4485414,13278.80,235.12
2812,2021-08-08,Los Angeles,California,1327058,24786.00,6037,CA,County,10039107,13218.88,246.89
...,...,...,...,...,...,...,...,...,...,...,...
1527155,2021-08-08,Wheeler,Oregon,53,1.00,41069,OR,County,1332,3978.98,75.08
1527455,2021-08-08,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1527724,2021-08-08,Esmeralda,Nevada,41,0.00,32009,NV,County,873,4696.45,0.00
1527989,2021-08-08,Loving,Texas,3,0.00,48301,TX,County,169,1775.15,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
565,2021-08-08,Snohomish,Washington,44286,625.00,53061,WA,County,822083,5387.05,76.03,76.03,5387.05
1128,2021-08-08,Cook,Illinois,568822,11147.00,17031,IL,County,5150233,11044.59,216.44,216.44,11044.59
1690,2021-08-08,Orange,California,287661,5153.00,6059,CA,County,3175692,9058.21,162.26,162.26,9058.21
2251,2021-08-08,Maricopa,Arizona,595609,10546.00,4013,AZ,County,4485414,13278.80,235.12,235.12,13278.80
2812,2021-08-08,Los Angeles,California,1327058,24786.00,6037,CA,County,10039107,13218.88,246.89,246.89,13218.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1527155,2021-08-08,Wheeler,Oregon,53,1.00,41069,OR,County,1332,3978.98,75.08,75.08,3978.98
1527455,2021-08-08,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1527724,2021-08-08,Esmeralda,Nevada,41,0.00,32009,NV,County,873,4696.45,0.00,0.00,4696.45
1527989,2021-08-08,Loving,Texas,3,0.00,48301,TX,County,169,1775.15,0.00,0.00,1775.15


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
976914,2021-08-08,Hale,Alabama,2396,79.00,1065,AL,County,14651,16353.83,539.21,539.21,16353.83,1
1063809,2021-08-08,Clarke,Alabama,3823,63.00,1025,AL,County,23622,16184.07,266.70,266.70,16184.07,2
763581,2021-08-08,Lowndes,Alabama,1474,55.00,1085,AL,County,9726,15155.25,565.49,565.49,15155.25,3
567516,2021-08-08,Franklin,Alabama,4723,81.00,1059,AL,County,31362,15059.63,258.27,258.27,15059.63,4
676741,2021-08-08,Etowah,Alabama,15315,372.00,1055,AL,County,102268,14975.36,363.75,363.75,14975.36,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932595,2021-08-08,Sublette,Wyoming,874,9.00,56035,WY,County,9831,8890.25,91.55,91.55,8890.25,19
932096,2021-08-08,Converse,Wyoming,1182,20.00,56009,WY,County,13822,8551.58,144.70,144.70,8551.58,20
1160141,2021-08-08,Lincoln,Wyoming,1550,14.00,56023,WY,County,19830,7816.44,70.60,70.60,7816.44,21
1292257,2021-08-08,Niobrara,Wyoming,175,3.00,56027,WY,County,2356,7427.84,127.33,127.33,7427.84,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
763581,2021-08-08,Lowndes,Alabama,1474,55.00,1085,AL,County,9726,15155.25,565.49,565.49,15155.25,3,1
976914,2021-08-08,Hale,Alabama,2396,79.00,1065,AL,County,14651,16353.83,539.21,539.21,16353.83,1,2
823698,2021-08-08,Greene,Alabama,1002,38.00,1063,AL,County,8111,12353.59,468.50,468.50,12353.59,37,3
323850,2021-08-08,Walker,Alabama,8008,291.00,1127,AL,County,63521,12606.85,458.12,458.12,12606.85,31,4
674733,2021-08-08,Clay,Alabama,1694,59.00,1027,AL,County,13235,12799.40,445.79,445.79,12799.40,27,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160141,2021-08-08,Lincoln,Wyoming,1550,14.00,56023,WY,County,19830,7816.44,70.60,70.60,7816.44,21,19
1131054,2021-08-08,Uinta,Wyoming,2498,14.00,56041,WY,County,20226,12350.44,69.22,69.22,12350.44,4,20
821697,2021-08-08,Hot Springs,Wyoming,394,3.00,56017,WY,County,4413,8928.17,67.98,67.98,8928.17,18,21
322326,2021-08-08,Teton,Wyoming,3960,11.00,56039,WY,County,23464,16876.92,46.88,46.88,16876.92,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,76.03,5387.05,20,24
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,76.03,5387.05,20,24
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,76.03,5387.05,20,24
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,76.03,5387.05,20,24
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,76.03,5387.05,20,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1524671,2021-08-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1524672,2021-08-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1524673,2021-08-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1524674,2021-08-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
972862,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,539.21,16353.83,2,1,1.00
972863,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,539.21,16353.83,2,1,0.00
972864,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,539.21,16353.83,2,1,0.00
972865,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,539.21,16353.83,2,1,0.00
972866,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,539.21,16353.83,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179705,2021-08-04,Crook,Wyoming,482,13.00,56011,WY,County,7584,6355.49,171.41,171.41,6355.49,7,23,0.00
1179706,2021-08-05,Crook,Wyoming,482,13.00,56011,WY,County,7584,6355.49,171.41,171.41,6355.49,7,23,0.00
1179707,2021-08-06,Crook,Wyoming,482,13.00,56011,WY,County,7584,6355.49,171.41,171.41,6355.49,7,23,0.00
1179708,2021-08-07,Crook,Wyoming,482,13.00,56011,WY,County,7584,6355.49,171.41,171.41,6355.49,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
760523,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,15155.25,1,3,1.00,0.00
760524,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,15155.25,1,3,0.00,0.00
760525,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,15155.25,1,3,0.00,0.00
760526,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,15155.25,1,3,0.00,0.00
760527,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,15155.25,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758013,2021-08-04,Albany,Wyoming,4805,18.00,56001,WY,County,38880,12358.54,46.30,46.30,12420.27,23,3,7.00,0.00
758014,2021-08-05,Albany,Wyoming,4820,18.00,56001,WY,County,38880,12397.12,46.30,46.30,12420.27,23,3,15.00,0.00
758015,2021-08-06,Albany,Wyoming,4829,18.00,56001,WY,County,38880,12420.27,46.30,46.30,12420.27,23,3,9.00,0.00
758016,2021-08-07,Albany,Wyoming,4829,18.00,56001,WY,County,38880,12420.27,46.30,46.30,12420.27,23,3,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-08-08') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
386810,2021-08-08,Imperial,California,32388,744.00,6025,CA,County,181215,17872.69,410.56,410.56,17872.69,1,2,87.00,0.00
2812,2021-08-08,Los Angeles,California,1327058,24786.00,6037,CA,County,10039107,13218.88,246.89,246.89,13218.88,2,5,3016.00,17.00
192212,2021-08-08,San Bernardino,California,316408,5295.00,6071,CA,County,2180085,14513.56,242.88,242.88,14513.56,3,4,1707.00,11.00
678701,2021-08-08,Inyo,California,1482,39.00,6027,CA,County,18039,8215.53,216.20,216.20,8215.53,4,31,0.00,1.00
85761,2021-08-08,Stanislaus,California,67115,1099.00,6099,CA,County,550660,12188.10,199.58,199.58,12188.10,5,9,0.00,0.00
68185,2021-08-08,San Joaquin,California,82989,1474.00,6077,CA,County,762148,10888.83,193.40,193.40,10888.83,6,15,618.00,0.00
39642,2021-08-08,Riverside,California,322894,4667.00,6065,CA,County,2470546,13069.74,188.91,188.91,13069.74,7,7,1911.00,0.00
106398,2021-08-08,Tulare,California,61481,854.00,6107,CA,County,466195,13187.83,183.19,183.19,13187.83,8,6,244.00,0.00
614837,2021-08-08,Merced,California,33761,488.00,6047,CA,County,277680,12158.24,175.74,175.74,12158.24,9,10,0.00,0.00
38602,2021-08-08,Fresno,California,116539,1747.00,6019,CA,County,999101,11664.39,174.86,174.86,11664.39,10,12,642.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1443535,2021-08-08,Lassen,California,5935,26.00,6035,CA,County,30573,19412.55,85.04,85.04,19412.55,35,1,0.00,0.00
386810,2021-08-08,Imperial,California,32388,744.00,6025,CA,County,181215,17872.69,410.56,410.56,17872.69,1,2,87.00,0.00
823640,2021-08-08,Kings,California,25371,252.00,6031,CA,County,152940,16588.86,164.77,164.77,16588.86,11,3,142.00,0.00
192212,2021-08-08,San Bernardino,California,316408,5295.00,6071,CA,County,2180085,14513.56,242.88,242.88,14513.56,3,4,1707.00,11.00
2812,2021-08-08,Los Angeles,California,1327058,24786.00,6037,CA,County,10039107,13218.88,246.89,246.89,13218.88,2,5,3016.00,17.00
106398,2021-08-08,Tulare,California,61481,854.00,6107,CA,County,466195,13187.83,183.19,183.19,13187.83,8,6,244.00,0.00
39642,2021-08-08,Riverside,California,322894,4667.00,6065,CA,County,2470546,13069.74,188.91,188.91,13069.74,7,7,1911.00,0.00
241821,2021-08-08,Kern,California,115007,1429.00,6029,CA,County,900202,12775.69,158.74,158.74,12775.69,13,8,0.00,0.00
85761,2021-08-08,Stanislaus,California,67115,1099.00,6099,CA,County,550660,12188.10,199.58,199.58,12188.10,5,9,0.00,0.00
614837,2021-08-08,Merced,California,33761,488.00,6047,CA,County,277680,12158.24,175.74,175.74,12158.24,9,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
386810,2021-08-08,Imperial,California,32388,744.00,6025,CA,County,181215,17872.69,410.56,410.56,17872.69,1,2,87.00,0.00
2812,2021-08-08,Los Angeles,California,1327058,24786.00,6037,CA,County,10039107,13218.88,246.89,246.89,13218.88,2,5,3016.00,17.00
192212,2021-08-08,San Bernardino,California,316408,5295.00,6071,CA,County,2180085,14513.56,242.88,242.88,14513.56,3,4,1707.00,11.00
678701,2021-08-08,Inyo,California,1482,39.00,6027,CA,County,18039,8215.53,216.20,216.20,8215.53,4,31,0.00,1.00
85761,2021-08-08,Stanislaus,California,67115,1099.00,6099,CA,County,550660,12188.10,199.58,199.58,12188.10,5,9,0.00,0.00
68185,2021-08-08,San Joaquin,California,82989,1474.00,6077,CA,County,762148,10888.83,193.40,193.40,10888.83,6,15,618.00,0.00
39642,2021-08-08,Riverside,California,322894,4667.00,6065,CA,County,2470546,13069.74,188.91,188.91,13069.74,7,7,1911.00,0.00
106398,2021-08-08,Tulare,California,61481,854.00,6107,CA,County,466195,13187.83,183.19,183.19,13187.83,8,6,244.00,0.00
614837,2021-08-08,Merced,California,33761,488.00,6047,CA,County,277680,12158.24,175.74,175.74,12158.24,9,10,0.00,0.00
38602,2021-08-08,Fresno,California,116539,1747.00,6019,CA,County,999101,11664.39,174.86,174.86,11664.39,10,12,642.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5082,08/04/21,Lassen,5905,26.00,19314.43,85.04,35,1,7.00,5.00
5083,08/05/21,Lassen,5935,26.00,19412.55,85.04,35,1,30.00,0.00
5084,08/06/21,Lassen,5935,26.00,19412.55,85.04,35,1,0.00,0.00
5085,08/07/21,Lassen,5935,26.00,19412.55,85.04,35,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5168,08/04/21,Fresno,115322,1746.00,11542.58,174.76,10,12,261.00,2.00
5169,08/05/21,Fresno,115577,1747.00,11568.10,174.86,10,12,255.00,1.00
5170,08/06/21,Fresno,115897,1747.00,11600.13,174.86,10,12,320.00,0.00
5171,08/07/21,Fresno,115897,1747.00,11600.13,174.86,10,12,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)